<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/preprocessing/SOLYS_to_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hier werden die Messwerte des SOLYS-Datenloggers in einen Dataframe umgewandelt:

**Dokumentenname: SOLYS_to_DF.ipynb**

Hier werden die Daten des SOLYS-Sun-Tracker in einen Dataframe umgewandelt:

## Import der benötigten Module und allgemeine Settings:

In [1]:
%matplotlib inline

In [2]:
# Import der benötigten Module
import os
import pandas as pd
import matplotlib.pyplot  as plt

from google.colab import drive

In [3]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Pfad zum Ordner der Drive in dem die Messdaten gespeichert sind
# Import:
folder_import = f'/content/drive/My Drive/Colab_Notebooks/SOLYS_Messdaten'

# Export:
folder_export = f'/content/drive/My Drive/Colab_Notebooks/SOLYS_CSV'
name_export = 'Solys_CSV'

## Daten einlesen:

In [5]:
# Funktion zum Laden aller Dateien in einem Google Drive-Ordner
def load_data_from_drive_folder(folder_path, header_lines=7, encoding = 'cp1252'):

    # Liste, um alle DataFrames zu speichern
    dfs = []

    # Durchlaufen alle Dateien im angegebenen Ordner
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Überprüfe, ob es eine Datei ist und lade sie
        if os.path.isfile(file_path):
            df = pd.read_csv(file_path, skiprows=header_lines, sep = '\t', encoding=encoding)
            dfs.append(df)

    # Kombinieren aller DataFrames
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

In [6]:
# Aufruf der Funktion zum Einlesen der Messdaten
df = load_data_from_drive_folder(folder_import)

## Löschen und umbenennen von Spalten:

In [7]:
# Löschen von nicht benötigten Splaten
df_Solys = df.drop(columns = ['Unit', 'Unnamed: 10', ' °C', 'V', ' °C.1', 'Unnamed: 9'], axis=1)

In [8]:
# Umbenennen der Splatennamen
df_Solys.columns = ['Datetime', 'Glo', 'Dif', 'Glo_SPLite', 'Dir', 'Temp']

In [9]:
# Löschen aller NaN Zeilen
df_Solys = df_Solys.dropna()

# Datetime-Spalte als Index setzen
df_Solys.set_index('Datetime', inplace = True)

# Index in Datetime umwandeln
df_Solys.index = pd.to_datetime(df_Solys.index, format='%d.%m.%Y %H:%M:%S')

In [10]:
# Dateityp der Spalten und Anzahl der Nullwerte
# print(df_Solys.info(), '\n --------------------- \n', df_Solys.isnull().sum(), df_Solys)

## Messungen in die richtige Reihenfolge bringen:

In [11]:
# Sotieren des Index in chronologischer Reihenfolge
df_Solys.sort_index(ascending=True, inplace = True)

In [12]:
# Prüfen, ob die Messdaten in chronologischer Reihenfolge sind
chronologisch = df_Solys.index.is_monotonic_increasing
print(f"Der Index ist in chronologischer Reihenfolge: {chronologisch}")

Der Index ist in chronologischer Reihenfolge: True


## Umformen der Trennzeichen:

In [13]:
for column in df_Solys[['Glo','Dif','Glo_SPLite','Dir','Temp']]:
    try:
        df_Solys[column] = df_Solys[column].str.replace(',', '.').astype(float)
    except ValueError:
        print(f"Spalte {column} konnte nicht in float umgewandelt werden.")

## Prüfen, ob die Daten zusammen hängen:

In [14]:
# Allgemeine Informationen
# round(df_Solys.describe(), 2).T

In [15]:
def pruefe_zeitlichen_abstand(df, min_abstand):
    ergebnisse = []

    for datum, gruppe in df.groupby(df.index.date):
        gruppe = gruppe.sort_index()
        abstaende = gruppe.index.to_series().diff().dropna()

        # Finden von Zeitstempeln, bei denen der Abstand zu klein ist
        zu_kleine_abstaende = abstaende[abstaende < min_abstand].index

        # Für jeden zu kleinen Abstand, speichern des Datums und der Uhrzeit mit der Bedingung False
        for zeit in zu_kleine_abstaende:
            ergebnisse.append({'Datum': datum, 'Uhrzeit': zeit.time(), 'Erfuellt': False})

        # Für jeden richtigen Abstand, speichern des Datums und der Uhrzeit mit der Bedingung True
        if zu_kleine_abstaende.empty:
            ergebnisse.append({'Datum': datum, 'Uhrzeit': None, 'Erfuellt': True})

    ergebnis_df = pd.DataFrame(ergebnisse)
    ergebnis_df['Datum'] = pd.to_datetime(ergebnis_df['Datum'])

    return ergebnis_df

In [16]:
# Aufruf der Funktion um zu prüfen, ob die Daten zusammenhängend sind
ergebnis_df = pruefe_zeitlichen_abstand(df_Solys, pd.Timedelta('10S'))
print(len(pd.unique(df_Solys.index.date)), len(ergebnis_df[ergebnis_df['Erfuellt']]))

504 499


In [17]:
# Tage an denen die Messdaten nicht zusammenhängen finden
# print(ergebnis_df[~ergebnis_df['Erfuellt']])

***Tage mit fehlenden Daten:***
    
         Datum    	    Erfuellt
1.   96  2022-08-08     False
2.   97  2022-08-09     False
3.   98  2022-08-10     False
4.   99  2022-08-11     False
5.   100 2022-08-12     False

## Plotten der fehlenden Messdaten:

In [18]:
df_plot = df_Solys

In [19]:
def plot_selected_days(df, date_column, value_columns, selected_dates):
    # Stelle sicher, dass das Datum in den DataFrames korrekt formatiert ist
    df_func = df

    df_func[date_column] = pd.to_datetime(df_func[date_column])
    selected_dates = pd.to_datetime(selected_dates)

    # print(selected_dates.dtype, df_func[date_column].dtype)

    # Gruppiere den DataFrame nach dem Datum
    grouped = df_func.groupby(df_func.index.date)

    # Erstelle für jeden ausgewählten Tag ein Plot
    for date, group in grouped:
        for selected_dates in date:
            print(date)

In [20]:
# plot_selected_days(df_plot, 'date', ['Glo', 'Dir'], ['2022-08-11', '2022-08-12'])
# ergebnis_df['Datum'][~ergebnis_df['Erfuellt']]

## Daten reduzieren:

Daten reduzieren damit die Auflösung mit der des Spektrometers überein stimmt:

In [21]:
# Durchschnittswerte für 2 Min. berechnen
df_Solys_mean = df_Solys.resample('2T').mean()

## Abspeichern der Messdaten als CSV in der Google-Drive

In [22]:
def save_dataframe_to_drive(dataframe, folder_path, filename):

    # Erstelle den vollen Pfad zur Datei
    full_path = os.path.join(folder_path, filename)

    # Speichere den DataFrame als CSV-Datei auf Google Drive
    dataframe.to_csv(full_path, index=True)

In [23]:
save_dataframe_to_drive(df_Solys_mean, folder_export, name_export)

In [24]:
df_Solys_mean

,Glo,Dif,Glo_SPLite,Dir,Temp
Datetime,,,,,
2022-04-25 08:06:00,245.833333,246.866667,259.233333,2.400000,10.766667
2022-04-25 08:08:00,237.058333,237.666667,249.200000,2.350000,10.658333
2022-04-25 08:10:00,206.466667,206.675000,217.633333,2.316667,10.058333
2022-04-25 08:12:00,188.858333,189.400000,200.900000,2.316667,10.608333
2022-04-25 08:14:00,186.991667,187.616667,199.800000,2.208333,10.391667
...,...,...,...,...,...
2023-11-09 08:10:00,9.975000,10.500000,11.266667,1.400000,8.683333
2023-11-09 08:12:00,10.233333,10.741667,11.291667,1.400000,8.708333
2023-11-09 08:14:00,10.775000,11.275000,11.691667,1.416667,8.716667
